In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from erlab.interactive.exampledata import generate_data

In [ ]:
%config InlineBackend.figure_formats = ["svg", "pdf"]
plt.rcParams["figure.dpi"] = 96
plt.rcParams["image.cmap"] = "viridis"

xr.set_options(display_expand_data=False)
nb_execution_mode = "cache"

Let's start by defining a model function and the data to fit.

In [ ]:
def poly1(x, a, b):
    return a * x + b

# Generate some toy data
x = np.linspace(0, 10, 20)
y = poly1(x, 1, 2)

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(1)
yerr = np.full_like(x, 0.5)
y = rng.normal(y, yerr)

In [ ]:
import lmfit

model = lmfit.Model(poly1)
params = model.make_params(a=1.0, b=2.0)
result = model.fit(y, x=x, params=params, weights=1 / yerr)

result.plot()
result

By passing dictionaries to `make_params`, we can set the initial values of the parameters and also set the bounds for the parameters.

In [ ]:
model = lmfit.Model(poly1)
params = model.make_params(
    a=dict(value=1.0, min=0.0),
    b=dict(value=2.0, vary=False),
)
result = model.fit(y, x=x, params=params, weights=1 / yerr)
_ = result.plot()

In [ ]:
result.params

In [ ]:
result.params["a"].value, result.params["a"].stderr

The parameters can also be retrieved in a form that allows easy error propagation calculation, enabled by the [uncertainties](https://github.com/lmfit/uncertainties) package.

In [ ]:
a_uvar = result.uvars["a"]
print(a_uvar)
print(a_uvar**2)

In [ ]:
from lmfit.models import LinearModel, GaussianModel

# Generate toy data
x = np.linspace(0, 10, 100)
y = -0.1 * x + 2 + 3 * np.exp(-((x - 5) ** 2) / (2 * 1 ** 2))

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(1)
yerr = np.full_like(x, 0.4)
y = rng.normal(y, yerr)

# Plot the data
plt.errorbar(x, y, yerr, fmt="o")

A composite model can be created by adding multiple models together.

In [ ]:
model = LinearModel() + GaussianModel()
params = model.make_params(slope=-0.1, center=5.0, sigma=dict(value=0.1, min=0))
params

In [ ]:
result = model.fit(y, x=x, params=params, weights=1 / yerr)
result.plot()
result

How about multiple gaussian peaks? Since the parameter names overlap between the models, we must use the `prefix` argument to distinguish between them.

In [ ]:
model = LinearModel() + GaussianModel(prefix="p0_") + GaussianModel(prefix="p1_")
model.make_params()

In [ ]:
from erlab.analysis.fit.models import MultiPeakModel

model = MultiPeakModel(npeaks=1, peak_shapes=["gaussian"], fd=False, convolve=False)
params = model.make_params(p0_center=5.0, p0_width=0.2, p0_height=3.0)
params

In [ ]:
result = model.fit(y, x=x, params=params, weights=1 / yerr)
_ = result.plot()

We can also plot components.

In [ ]:
comps = result.eval_components()
plt.errorbar(x, y, yerr, fmt="o", zorder=-1, alpha=0.3)
plt.plot(x, result.eval(), label="Best fit")
plt.plot(x, comps["MultiPeakFunction_p0"], "--", label="Peak")
plt.plot(x, comps["MultiPeakFunction_bkg"], "--", label="Background")
plt.legend()

Now, let us try fitting MDCs cut from simulated data with multiple Lorentzian peaks, convolved with a common instrumental resolution.

In [ ]:
data = generate_data(bandshift=-0.2, count=30).T
cut = data.qsel(ky=0.3)
cut.qplot(colorbar=True)

In [ ]:
mdc = cut.qsel(eV=0.0)
mdc.plot()

First, we define the model and set the initial parameters.

In [ ]:
from erlab.analysis.fit.models import MultiPeakModel

model = MultiPeakModel(npeaks=4, peak_shapes=["lorentzian"], fd=False, convolve=True)

params = model.make_params(
    p0_center=-0.59,
    p1_center=-0.45,
    p2_center=0.45,
    p3_center=0.6,
    p0_width=0.02,
    p1_width=0.02,
    p2_width=0.02,
    p3_width=0.02,
    lin_bkg=dict(value=0.0, vary=False),
    const_bkg=0.0,
    resolution=0.03,
)
params

Then, we can fit the model to the data:

In [ ]:
result = model.fit(mdc, x=mdc.kx, params=params)
result.plot()
result

In [ ]:
from erlab.analysis.fit.minuit import Minuit
from erlab.analysis.fit.models import MultiPeakModel

model = MultiPeakModel(npeaks=4, peak_shapes=["lorentzian"], fd=False, convolve=True)

m = Minuit.from_lmfit(
    model,
    mdc,
    mdc.kx,
    p0_center=-0.59,
    p1_center=-0.45,
    p2_center=0.45,
    p3_center=0.6,
    p0_width=0.02,
    p1_width=0.02,
    p2_width=0.02,
    p3_width=0.02,
    p0_height=1500,
    p1_height=50,
    p2_height=50,
    p3_height=1500,
    lin_bkg=dict(value=0.0, vary=False),
    const_bkg=0.0,
    resolution=0.03,
)

m.migrad()
m.minos()
m.hesse()

In [ ]:
m.interactive()